In [1]:
from narupa.app.client import NarupaClient
from imd_openmm import openmm_ase_frame_server, start_imd
from narupa.ase.converter import frame_data_to_ase
from narupa.trajectory.frame_data import (
    FrameData, POSITIONS, ELEMENTS, TYPES, MissingDataError,
)
from narupa.imd.interaction import Interaction
import numpy as np
import nglview
import MDAnalysis as mda
import matplotlib.pyplot as plt
from queue import Queue

## Start an IMD client

In [2]:
class NGLClient(NarupaClient):
    def __init__(self, *args, update_callback=None, **kwargs):
        super().__init__(*args, **kwargs)
        self._view = None
        self.update_callback = update_callback
    
    @property
    def view(self):
        if self._view is None:
            atoms = frame_data_to_ase(
                client.first_frame,
                topology=True,
                positions=False,
            )
            atoms.set_positions(
                np.array(client.latest_frame.particle_positions) * 10
            )
            self._view = nglview.show_ase(atoms)
        return self._view

    def _on_frame_received(self, frame_index:int, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates(
            {0: np.array(self.latest_frame.particle_positions) * 10}
        )
        if self.update_callback is not None:
            self.update_callback(self.universe)

In [20]:
client = NGLClient()

In [22]:
client.latest_frame.raw

values {
  key: "energy.kinetic"
  value {
    number_value: 0.3310391640760655
  }
}
arrays {
  key: "atom.id"
  value {
    string_values {
      values: "0"
      values: "1"
      values: "2"
      values: "3"
      values: "4"
      values: "5"
      values: "6"
      values: "7"
    }
  }
}
arrays {
  key: "atom.residue"
  value {
    index_values {
      values: 0
      values: 0
      values: 0
      values: 0
      values: 0
      values: 0
      values: 0
      values: 0
    }
  }
}
arrays {
  key: "bond"
  value {
    index_values {
      values: 0
      values: 1
      values: 0
      values: 2
      values: 0
      values: 3
      values: 0
      values: 4
      values: 1
      values: 5
      values: 1
      values: 6
      values: 1
      values: 7
    }
  }
}
arrays {
  key: "particle.element"
  value {
    index_values {
      values: 6
      values: 6
      values: 1
      values: 1
      values: 1
      values: 1
      values: 1
      values: 1
    }
  }
}
arrays {
 

In [18]:
positions = np.array(client.latest_frame.particle_positions) * 10
start_point = positions[0, :]
first_anchor = positions[0] - np.array([0.1,0,0])

In [19]:
anchor1 = Interaction(position=first_anchor, scale=1000, particles=(0,), interaction_type='spring')
anchor1_id = client.start_interaction()
client.update_interaction(anchor1_id, anchor1)

In [7]:
client.view

NGLWidget()

In [8]:
client.view.shape.add_sphere(list(start_point), [1, 0, 0], 0.5)